In [1]:
import os
import sys
import tensorflow as tf

# we need to append the root directory to the path to import data.py\n",
sys.path.append('..')
import data
import utils
import model

%load_ext autoreload
%autoreload 2

In [2]:
params = utils.yaml_to_dict('../config.yml')
params['data_dir_images'] = os.path.join('..', params['data_dir_images'])
params['data_dir'] = os.path.join('..', params['data_dir'])

## Model Summary

In [3]:
inputs = tf.keras.layers.Input(shape=(224, 224, 3))
architecture = model.MobilNet_Architecture(width_multiplier=1)
x = architecture(inputs, training=False)
architecture.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 112, 112, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 112, 112, 32)      128       
_________________________________________________________________
activation (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
depthwise_separable_convolut (None, 112, 112, 64)      3840      
_________________________________________________________________
depthwise_separable_convolut (None, 56, 56, 128)       13824     
_________________________________________________________________
depthwise_separable_convolut (None, 56, 56, 128)       35200     
_________________________________________________________________
depthwise_separable_convolut (None, 28, 28, 256)       52224     
__________